In [3]:
from flask import Flask, render_template, request, redirect, url_for
import os
import subprocess

app = Flask(__name__)
app.jinja_env.globals.update(zip=zip)

workflow = []

@app.route('/', methods=['GET', 'POST'])
def index():
    results = request.args.getlist('results') or [None] * len(workflow)
    outputs = request.args.getlist('outputs') or [None] * len(workflow)
    return render_template('index.html', workflow=workflow, results=results, outputs=outputs)

@app.route('/execute_workflow', methods=['POST'])
def execute_workflow():
    global workflow
    results = []
    outputs = []
    for command in workflow:
        try:
            result = subprocess.run(['powershell.exe', '-Command', command], check=True, stdout=subprocess.PIPE, text=True)
            results.append('success')
            outputs.append(result.stdout.strip())
        except subprocess.CalledProcessError as e:
            results.append('error')
            outputs.append(e.stdout.strip() if e.stdout else 'Error occurred during execution')
    return redirect(url_for('index', results=results, outputs=outputs))

@app.route('/delete_workflow', methods=['POST'])
def delete_workflow():
    global workflow
    workflow = []
    return redirect(url_for('index'))

@app.route('/create_step', methods=['POST'])
def create_step():
    command = request.form.get('command')
    if command:
        workflow.append(command)
    return redirect(url_for('index'))

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 5000, app)

 * Running on http://localhost:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Mar/2023 22:48:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2023 22:48:16] "POST /create_step HTTP/1.1" 302 -
127.0.0.1 - - [26/Mar/2023 22:48:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2023 22:48:18] "POST /create_step HTTP/1.1" 302 -
127.0.0.1 - - [26/Mar/2023 22:48:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2023 22:48:22] "POST /execute_workflow HTTP/1.1" 302 -
127.0.0.1 - - [26/Mar/2023 22:48:22] "GET /?results=success&results=success&outputs=Directory%3A+C%3A%5CUsers%5Cmarkl%5COneDrive%5CDesktop%5CHoneyBatch%5CHoneyBatch%0A%0A%0AMode+++++++++++++++++LastWriteTime+++++++++Length+Name+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++%0A----+++++++++++++++++-------------+++++++++------+----+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++%0Ada---l++++++++26%2F03%2F2023+++++21%3A46++++++++++++++++.ipynb_checkpoints++++++++++++++++++++++++++++++++++++++++++++++++